# get list sorting from gemini

Install necessary packages

In [1]:
# !pip install -q -U google-generativeai
# !pip install python-dotenv

In [2]:
import os
from dotenv import load_dotenv
import random
import time
import json
import google.generativeai as genai

### load model

In [3]:
load_dotenv()
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

In [4]:
gemini = genai.GenerativeModel('gemini-pro')

### load data and create initial prompt

In [5]:
with open('../../data/lists.json','r') as file:
    data = json.load(file)

# create batches, batch sizes are heuristics and I tried out several
# batch sizes, although input token limit is 30,720 tokens, output token limit is 2048
# which restricts the batch size
size_dict = {'lists_size_3_pos':50,
             'lists_size_3_neg':50,
             'lists_size_5_pos':40,
             'lists_size_5_neg':40,
             'lists_size_10_pos':25,
             'lists_size_10_neg':25}

batch_prompts = []

for list in data:
    list_ = data[list]

    # get batch size for the list
    BATCH_SIZE = size_dict[list]

    index = 0
    while index < len(list_):
        batch = []
        while index < len(list_):
            batch_prompt = [f"Sort the numbers in the list {list_[i]}." for i in range(index, min(index+BATCH_SIZE, len(list_)))]
            batch_prompts.append(batch_prompt)
            index += BATCH_SIZE


print("Number of batches:", len(batch_prompts))
print("Number of prompts in each batch:", [len(batch) for batch in batch_prompts])


Number of batches: 86
Number of prompts in each batch: [50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 20, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 20, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25]


In [12]:
# give a prompt once and then generate the response,
# and then use this prompt+response directly instead of doing everytime

promt_saying_you_are_a_math_assistant = '''You are a math assistant. I will ask you to sort the numbers in a list. Please answer in the correct format. For example, if I ask 'Sort the numbers in the list [3, 1, 4]', you should answer '[1, 3, 4]'. Each question is in a separate line. Please return each answer in a separate line. Do you understand ?'''
Initial_messages = [{'role':'user', 'parts': [promt_saying_you_are_a_math_assistant] }]

# get response from the model and then append it to the prompt
Initial_response = gemini.generate_content(Initial_messages)                                   
Initial_messages.append(Initial_response.candidates[0].content)

In [13]:
print(Initial_messages)

[{'role': 'user', 'parts': ["You are a math assistant. I will ask you to sort the numbers in a list. Please answer in the correct format. For example, if I ask 'Sort the numbers in the list [3, 1, 4]', you should answer '[1, 3, 4]'. Each question is in a separate line. Please return each answer in a separate line. Do you understand ?"]}, parts {
  text: "Yes, I understand. I am ready to assist you with sorting the numbers in a list."
}
role: "model"
]


In [14]:
import pickle

# save the initial messages to a file, in case we need to restart the conversation, initial message should be consistent
# across all batches

with open('initial_messages_sort.pkl', 'wb') as file:
    pickle.dump(Initial_messages, file)

### conduct experiment of sorting

In [15]:
# path to save responses
response_file = 'gemini_response/list_sort_response.txt'

# request per minute limit for free api
rpm = 60

In [16]:
for i, batch_prompt in enumerate(batch_prompts):

    # to start from a specific batch number, give printed_batch_num -1 here
    batch_no_to_start_from = 0 
    if i < batch_no_to_start_from:
        continue

    # print status
    print(f"Sending batch {i+1} of {len(batch_prompts)}")

    num_prompts = len(batch_prompt)

    # create prompt for the batch
    curr_prompt = "".join([f"{prompt} \n " for prompt in batch_prompt])

    
    # create a complete prompt to send to the model
    complete_promt = Initial_messages.copy()
    complete_promt.append({'role':'user', 'parts': [curr_prompt]})

    # get response from the model, candidate_count is 1 because we only need one response,
    # temperature is 0 because we need deterministic responses
    response = gemini.generate_content( complete_promt,
                                        generation_config= genai.GenerationConfig(
                                        candidate_count = 1,
                                        temperature = 0.0,
                                        )
                                )
    
    try:
        # extract text from the response
        model_responses = response.candidates[0].content.parts.__getitem__(0).text.split('\n')
    except IndexError:
        # if model gives no response, fill with 0
        print("Error in response")
        model_responses = ["0"]*num_prompts

    # save responses to file
    with open(response_file, 'a') as file:
        for model_response in model_responses:
            file.write(model_response + '\n')

    
    #print the prompt and response separated by space
    print('# of prompts: ',len(batch_prompt),'# of responses: ', len(model_responses))


    for prompt, response in zip(batch_prompt, model_responses):
        print(prompt, response, sep=" ->")
    
    
    print("Received responses for batch", i+1, "of", len(batch_prompts))
    print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")

    time.sleep(60//rpm) #to avoid rate limit errors

print("Responses saved to", response_file)

Sending batch 1 of 86
# of prompts:  50 # of responses:  50
Sort the numbers in the list [1, 0, 4]. ->[0, 1, 4]
Sort the numbers in the list [6, 0, 0]. ->[0, 0, 6]
Sort the numbers in the list [3, 8, 6]. ->[3, 6, 8]
Sort the numbers in the list [5, 4, 2]. ->[2, 4, 5]
Sort the numbers in the list [5, 9, 4]. ->[4, 5, 9]
Sort the numbers in the list [4, 9, 5]. ->[4, 5, 9]
Sort the numbers in the list [3, 1, 6]. ->[1, 3, 6]
Sort the numbers in the list [4, 1, 9]. ->[1, 4, 9]
Sort the numbers in the list [8, 3, 5]. ->[3, 5, 8]
Sort the numbers in the list [3, 9, 5]. ->[3, 5, 9]
Sort the numbers in the list [3, 8, 8]. ->[3, 8, 8]
Sort the numbers in the list [2, 8, 7]. ->[2, 7, 8]
Sort the numbers in the list [1, 6, 6]. ->[1, 6, 6]
Sort the numbers in the list [8, 4, 5]. ->[4, 5, 8]
Sort the numbers in the list [8, 2, 8]. ->[2, 8, 8]
Sort the numbers in the list [2, 8, 8]. ->[2, 8, 8]
Sort the numbers in the list [4, 3, 0]. ->[0, 3, 4]
Sort the numbers in the list [2, 2, 7]. ->[2, 2, 7]
Sort